In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from mpl_toolkits.axes_grid.inset_locator import mark_inset
frame_id = 0
block_region = pd.read_csv("block_region_"+str(frame_id)+".csv", sep=",")
st_nodes = pd.read_csv("st_node_"+str(frame_id)+".csv", sep=",")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3.09))
# axin = ax.inset_axes([0.1, 0.55, 0.35, 0.35])

# s-t nodes
for idx, row in st_nodes.iterrows():
    [s0, v0, a, t0, tf] = row[0:5]
    t = np.linspace(0, tf, 10)
    s = s0 + v0 * t + 0.5 * a * t ** 2
    t += t0
    [color, lw, zorder] = ["gray", 0.4, 1]
    if row[-2] == 1:
        [color, lw, zorder] = ["orangered", 1.0, 120]
    ax.plot(t, s, lw=lw, color=color, zorder=zorder, alpha=0.7)
#     axin.plot(t, s, lw=lw, color=color, zorder=zorder)

# block region
for idx, row in block_region.iterrows():
    x = row[0:5]
    y = row[5:10]
    ax.plot(x, y, color="b", lw=1)

# boundary
bound_color = "k"
t = np.linspace(0, 6.5, 100)
s_upper = st_nodes['s0'][0] + st_nodes['v0'][0] * t + 0.5 * 2 * t ** 2
ax.plot(t, s_upper, lw=1.5, color=bound_color)
# s_max = st_nodes['v0'][0] + st_nodes['v0'][0]**2 / 8
# ax.plot([2.5, 8], [s_max, s_max], lw=1.5, color=bound_color)
t = np.linspace(0, 5, 100)
s_lower = st_nodes['s0'][0] + st_nodes['v0'][0] * t - 0.5 * 4 * t ** 2
ax.plot(t, s_lower, lw=1.5, color=bound_color)

zoom_color = "darkblue"

# axin.set_xlim(2.5, 3.5)
# axin.set_ylim(36, 46)
# patch, pp1, pp2 = mark_inset(
#     ax, axin, loc1=3, loc2=1, fc="none", ec=zoom_color, lw=1, zorder=100
# )
# pp1.loc2 = 2
# pp2.loc1 = 4
# pp2.loc2 = 1

# ax.set_axis_off()
# axin.set_axis_off()

# fu.set_axis_color(axin, zoom_color)

tex_fonts = {
    # Use LaTeX to write all text
#     "text.usetex": True,
    "font.family": "Times New Roman",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    "font.size": 10,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
}
plt.rcParams.update(tex_fonts)
# ax.set_xticks(np.linspace(0, 8, 9))
# ax.set_yticks(np.linspace(0, 120, 60))
ax.grid(alpha=0.3)
ax.text(8, 5, "t(s)")
ax.text(-0.1, 110, "s(m)")

my_fc = "fuchsia"



ax.text(3.5, 8.1, "$a_{min}$", color="b")
ax.text(5, 110, "$a_{max}$", color="b")
ax.text(6.2, 14.2, "$v=0$", color="b")


fig.savefig(
    "../figure/st_node.pdf", format="pdf", bbox_inches="tight", pad_inches=0, dpi=300
)

plt.show()

# local trunction

In [1]:
import operator
import numpy as np
import matplotlib.pyplot as plt
# import awesome_plot.style.figsize_utils as fu
from mpl_toolkits.axes_grid.inset_locator import mark_inset
import matplotlib.patches as patches


class Node:
    def __init__(self, node=None, a=None, dt=None) -> None:
        if node == None:
            self.s0 = 0.0
            self.s = 0.0
            self.v0 = 0.0
            self.v = 0.0
            self.t0 = 0.0
            self.tf = 0.0
            self.a = 0.0
            self.dt = 0.0
            self.active = True
            self.J = 0.0
        else:
            self.s0 = node.s
            self.v0 = node.v
            self.t0 = node.tf
            self.s = self.s0 + self.v0 * dt + 0.5 * a * dt ** 2
            self.v = self.v0 + a * dt
            self.tf = self.t0 + dt
            self.a = a
            self.dt = dt
            self.active = False
            self.J = node.J + np.abs(a) * dt

    def sample(self):
        t = np.linspace(0, self.dt, 50)
        s = self.s0 + self.v0 * t + 0.5 * self.a * t ** 2
        t = t + self.t0
        return t, s


fig, ax = plt.subplots(1, 1, figsize=(5,3.09))
axin = ax.inset_axes([0.55, 0.05, 0.4, 0.9])
axin.set_xlim(1.8, 2.1)
axin.set_ylim(-1, 1)
axin.set_axis_off()

all_a = np.linspace(-4, 4, 13)

node_segs = [[Node()]]

for i in range(2):
    node_segs.append([])
    for node in node_segs[i]:
        if node.active:
            for a in all_a:
                node_segs[i + 1].append(Node(node, a, 1))
    node_segs[i + 1].sort(key=operator.attrgetter("s"))

    groups = [[]]
    last_s = node_segs[i + 1][0].s
    for node in node_segs[i + 1]:
        if node.s - last_s >= 0.1:
            groups.append([])
        groups[-1].append(node)
        last_s = node.s

    print(len(groups))
    for group in groups:
        if len(group) == 0:
            continue
        min_idx = 0
        min_J = 1e5
        for idx, node in enumerate(group):
            if node.J < min_J:
                min_J = node.J
                min_idx = idx
        group[min_idx].active = True

        for node in group:
            [t, s] = node.sample()
            if node.active:
                ax.plot(t, s, color="royalblue", lw=0.3)
                axin.plot(t, s, color="royalblue", lw=1)
                axin.add_patch(
                    patches.Ellipse(
                        (t[-1], s[-1]), width=0.06, height=0.28, fill=False, ec="tan"
                    )
                )
            else:
                ax.plot(t, s, color="gray", lw=0.3)
                axin.plot(t, s, color="gray", lw=0.8)

patch, pp1, pp2 = mark_inset(
    ax, axin, loc1=3, loc2=2, fc="none", ec="k", lw=1, zorder=100
)
# ax.set_aspect('equal', adjustable='box')
ax.set_xlim(0, 4)
ax.set_axis_off()

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "Times New Roman",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    "font.size": 8,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
}
plt.rcParams.update(tex_fonts)

ax.plot([1, 1], [-7.5, 7.5], "--", color="orange")
ax.text(0.2, 7, "round 1")
ax.text(1.2, 7, "round 2")
axin.text(2.04, -0.06, "$R_{g}$")

fig.savefig(
    "../figure/local_truncation.pdf",
    format="pdf",
    bbox_inches="tight",
    pad_inches=0,
    dpi=600,
)

plt.show()


/home/kin/anaconda3/envs/vectornet/lib/python3.7/site-packages/ipykernel_launcher.py:5: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  """


13
49


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../845f8a8cdff43e20ba665c5e0c349657.tex
LaTeX2e <2017-04-15>
Babel <3.18> and hyphenation patterns for 3 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.5 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 845f8a8cdff43e20ba665c5e0c349657.log.




RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../845f8a8cdff43e20ba665c5e0c349657.tex
LaTeX2e <2017-04-15>
Babel <3.18> and hyphenation patterns for 3 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.5 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 845f8a8cdff43e20ba665c5e0c349657.log.




<Figure size 360x222.48 with 1 Axes>

# backup

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8, 8))
# axin = ax.inset_axes([0.1, 0.55, 0.35, 0.35])

# s-t nodes
for idx, row in st_nodes.iterrows():
    [s0, v0, a, t0, tf] = row[0:5]
    t = np.linspace(0, tf, 10)
    s = s0 + v0 * t + 0.5 * a * t ** 2
    t += t0
    [color, lw, zorder] = ["gray", 0.4, 1]
    if row[-2] == 1:
        [color, lw, zorder] = ["orangered", 1.0, 120]
    ax.plot(t, s, lw=lw, color=color, zorder=zorder, alpha=0.7)
#     axin.plot(t, s, lw=lw, color=color, zorder=zorder)

# block region
for idx, row in block_region.iterrows():
    x = row[0:5]
    y = row[5:10]
    ax.plot(x, y, color="b", lw=1)

# boundary
bound_color = "k"
t = np.linspace(0, 6.5, 100)
s_upper = st_nodes['s0'][0] + st_nodes['v0'][0] * t + 0.5 * 2 * t ** 2
ax.plot(t, s_upper, lw=1.5, color=bound_color)
# s_max = st_nodes['v0'][0] + st_nodes['v0'][0]**2 / 8
# ax.plot([2.5, 8], [s_max, s_max], lw=1.5, color=bound_color)
t = np.linspace(0, 5, 100)
s_lower = st_nodes['s0'][0] + st_nodes['v0'][0] * t - 0.5 * 4 * t ** 2
ax.plot(t, s_lower, lw=1.5, color=bound_color)

zoom_color = "darkblue"

# axin.set_xlim(2.5, 3.5)
# axin.set_ylim(36, 46)
# patch, pp1, pp2 = mark_inset(
#     ax, axin, loc1=3, loc2=1, fc="none", ec=zoom_color, lw=1, zorder=100
# )
# pp1.loc2 = 2
# pp2.loc1 = 4
# pp2.loc2 = 1

# ax.set_axis_off()
# axin.set_axis_off()

# fu.set_axis_color(axin, zoom_color)

# tex_fonts = {
#     # Use LaTeX to write all text
# #     "text.usetex": True,
#     "font.family": "Times New Roman",
#     # Use 10pt font in plots, to match 10pt font in document
#     "axes.labelsize": 10,
#     "font.size": 10,
#     # Make the legend/label fonts a little smaller
#     "legend.fontsize": 8,
#     "xtick.labelsize": 8,
#     "ytick.labelsize": 8,
# }
# plt.rcParams.update(tex_fonts)
# ax.set_xticks(np.linspace(0, 8, 9))
# ax.set_yticks(np.linspace(0, 120, 60))
ax.grid(alpha=0.3)
ax.text(8, 5, "t(s)")
ax.text(-0.1, 110, "s(m)")

my_fc = "fuchsia"


# def draw_polygon(ax, polygon):
#     # ax.add_patch(Polygon(polygon, color="w", fill=True, alpha=1, zorder=100, lw=0.1))
#     ax.add_patch(
#         Polygon(
#             polygon,
#             closed=True,
#             color=my_fc,
#             fill=False,
#             hatch="\\\\\\\\",
#             lw=0.6,
#             zorder=101,
#         )
#     )
#     ax.add_patch(
#         Polygon(polygon, closed=True, color=my_fc, fill=False, lw=0.8, zorder=102)
#     )


ax.text(3.5, 8.1, "$a_{min}$", color="b")
ax.text(5, 110, "$a_{max}$", color="b")
ax.text(6.2, 14.2, "$v=0$", color="b")
# ax.add_patch(
#     Polygon(
#         [[5.2, 27], [7.9, 27], [7.9, 36], [5.2, 36]],
#         closed=True,
#         color="w",
#         fill=True,
#         lw=0.8,
#         zorder=2,
#     )
# )
# ax.text(5.2, 30, "convex region", color=my_fc, stretch='extra-condensed')

# ax.plot([4, 5, 5, 4, 4], [22.3, 22.3, 52, 43, 22.5], color="orange", linewidth=1)
# draw_polygon(ax, [[4, 22.3], [5, 22.3], [5, 52], [4, 43], [4, 22.5]])

# ax.arrow(2.5, 8.5, 1, 1)
# ax.annotate(
#     "$a_{max}$", xy=(3, 19), xytext=(1.5, 8.5), arrowprops=dict(width=3, headwidth=5)
# )

# fig.savefig(
#     "../figure/st_node.pdf", format="pdf", bbox_inches="tight", pad_inches=0, dpi=300
# )

plt.show()